In [3]:
import json
import os
import openai
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/qlin/opt/anaconda3/envs/python311/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
os.environ["OPENAI_API_KEY"] = "api"

os.environ['PINECONE_API_KEY'] = "api"

In [5]:
openai.api_key = "api"

In [6]:
from langchain.document_loaders import DirectoryLoader, TextLoader
loader = DirectoryLoader('/Users/qlin/Desktop/Fall2023/NoSQL/Project/Final_Project', glob='**/*.json', show_progress=True, loader_cls=TextLoader)

documents = loader.load()

100%|██████████| 1/1 [00:00<00:00, 34.04it/s]


In [3]:
loader = UnstructuredFileLoader(
    "/Users/qlin/Desktop/Fall2023/NoSQL/Project/Final_Project/filingMetadata.json"
    #post_processors=[clean_extra_whitespace],
)
documents = loader.load()

In [7]:
# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [8]:
# Pinecone Environment Variable
pinecone.init(      
	api_key='api',      
	environment='gcp-starter'      
)      
index = pinecone.Index('nosql-project')

In [9]:
limit = 3600
embed_model = "text-embedding-ada-002"

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    res = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Given the contextual information and not prior knowledge, answer the question. If the answer is not in the context, inform the user that you can't answer the question.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [7]:
# first let's make it simpler to get answers
def complete(prompt):
    # query text-davinci-003
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=400,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [10]:
query = "Which exchange is Apple's stock traded on"

In [11]:
# first we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
query_with_contexts

UnboundLocalError: cannot access local variable 'prompt' where it is not associated with a value

In [ ]:
# then we complete the context-infused query
complete(query_with_contexts)